In [1]:
import numpy as np
import os
import pandas as pd
import pickle as pkl
from utils import variants_to_onehot_seq, variants_to_seq
from Bio import SeqIO

In [2]:
# data_dir = "../Data/"
# output_data_dir = "data"

data_dir = "Encodings/tmp_data/"
output_data_dir = "Encodings/tmp_data/"

In [5]:
protein_variants_files = dict()
# List only folders in data folder
for folder in os.listdir(data_dir,):
    if os.path.isdir(os.path.join(data_dir, folder)):
        variants = []
        # List every file in folder with fasta extension
        for file in os.listdir(data_dir + folder):
            if file.endswith(".csv"):
                variants.append(file)
        protein_variants_files[folder] = variants

In [6]:
protein_variants_files

{'PABP_YEAST_Fields2013': ['PABP_YEAST_Fields2013_encoded.csv']}

In [8]:
for protein, variants_files in protein_variants_files.items():
    for variants_file in variants_files:
        
        df = pd.read_csv(os.path.join(data_dir, protein, variants_file), sep=";")
        fitness_df = df["y"]
        variants_df = df["variant"]
        
        # Get the wild type
        wild_type_file = os.path.join(data_dir, protein, protein+"_wt.fasta")
        wild_type = SeqIO.read(wild_type_file, "fasta")
        wild_type_seq = wild_type.seq
        print(wild_type.description)
        wilt_type_startpos = int(wild_type.description.lower().split("start: ")[1].split(",")[0])

        # Convert variants to one-hot encoded sequences
        X = variants_to_seq(variants_df, wild_type_seq, wilt_type_startpos)
        y = fitness_df.to_numpy()

        # Save to pickle
        # Create protein directory if it doesn't exist
        if not os.path.exists(os.path.join(output_data_dir, protein)):
            os.makedirs(os.path.join(output_data_dir, protein))
        pkl.dump(X, open(os.path.join(output_data_dir, protein, variants_file.split("_encoded")[0] + "_X.pkl"), "wb"))
        pkl.dump(y, open(os.path.join(output_data_dir, protein, variants_file.split("_encoded")[0] + "_y.pkl"), "wb"))

 pabp_yeast | start: 126, end: 200, length: 75 | OFFSET: 126


In [9]:
for protein, variants_files in protein_variants_files.items():
    for variants_file in variants_files:
        X = pkl.load(open(os.path.join(output_data_dir, protein, variants_file.split("_encoded")[0] + "_X.pkl"), "rb"))
        y = pkl.load(open(os.path.join(output_data_dir, protein, variants_file.split("_encoded")[0] + "_y.pkl"), "rb"))
        print(X.shape, y.shape)

AttributeError: 'list' object has no attribute 'shape'